<a href="https://colab.research.google.com/github/TonyWang126/NYU-Econometrics-2021/blob/main/Note_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Briefings**

1. Two packages:


*   https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
*   https://bashtage.github.io/linearmodels/doc/iv/examples/advanced-examples.html

2. Four variables:

Dependent Variables: Y
Endogenous Variable: X
Control variables: W
instruments: Z

3. Two regressions


*   First: X~Z
*   Second: Y~(estimated_X+W)





background readings: https://cryptobriefing.com/earning-20-apy-with-yield-app/

# Data Preparations

### Date Sources 
https://docs.defipulse.com/overview-of-api-endpoints

https://data-beta.defipulse.com/dashboard/egs

https://docs.defipulse.com/api-docs-by-provider/defi-pulse-data/total-value-locked/total-value-locked

### Import the Historical Data of Total Value Locked

In [ ]:
import pandas as pd
import requests
import json 
from datetime import datetime
TVL= requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory?resolution=history&project=all&api-key=05f8e579a2bec58c6fb86b5ad1d84b76b0267a190307cdf6f194960c4121')
TVL= TVL.json()
TVL=pd.DataFrame(TVL)
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI
0,1624806000,48470247455,2.626473e+07,164703.242424,8.749086e+06,2.674866e+09
1,1624752000,47980997651,2.620108e+07,165755.662931,8.740187e+06,2.689429e+09
2,1624665600,47965626508,2.641104e+07,163732.474282,8.746788e+06,2.701426e+09
3,1624579200,50994717087,2.560618e+07,164570.777150,8.682699e+06,2.692037e+09
4,1624492800,50101916045,2.539506e+07,162681.342310,8.509900e+06,2.663994e+09


### Epoch & Unix Timestamp Conversion Tools: 

https://www.epochconverter.com/

In [ ]:
import datetime
TVL['timestamp']=TVL['timestamp'].astype(int)
TVL.dtypes

timestamp      int64
tvlUSD         int64
tvlETH       float64
BTC          float64
ETH          float64
DAI          float64
dtype: object

In [ ]:
TVL['time']=TVL['timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).replace(tzinfo=datetime.timezone.utc))
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time
0,1624806000,48470247455,2.626473e+07,164703.242424,8.749086e+06,2.674866e+09,2021-06-27 15:00:00+00:00
1,1624752000,47980997651,2.620108e+07,165755.662931,8.740187e+06,2.689429e+09,2021-06-27 00:00:00+00:00
2,1624665600,47965626508,2.641104e+07,163732.474282,8.746788e+06,2.701426e+09,2021-06-26 00:00:00+00:00
3,1624579200,50994717087,2.560618e+07,164570.777150,8.682699e+06,2.692037e+09,2021-06-25 00:00:00+00:00
4,1624492800,50101916045,2.539506e+07,162681.342310,8.509900e+06,2.663994e+09,2021-06-24 00:00:00+00:00


In [ ]:
TVL['date'] = pd.to_datetime(TVL['time']).dt.date
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time,date
0,1624806000,48470247455,2.626473e+07,164703.242424,8.749086e+06,2.674866e+09,2021-06-27 15:00:00+00:00,2021-06-27
1,1624752000,47980997651,2.620108e+07,165755.662931,8.740187e+06,2.689429e+09,2021-06-27 00:00:00+00:00,2021-06-27
2,1624665600,47965626508,2.641104e+07,163732.474282,8.746788e+06,2.701426e+09,2021-06-26 00:00:00+00:00,2021-06-26
3,1624579200,50994717087,2.560618e+07,164570.777150,8.682699e+06,2.692037e+09,2021-06-25 00:00:00+00:00,2021-06-25
4,1624492800,50101916045,2.539506e+07,162681.342310,8.509900e+06,2.663994e+09,2021-06-24 00:00:00+00:00,2021-06-24


In [ ]:
TVL=TVL.drop(columns='time')
TVL['date']=pd.to_datetime(TVL['date'])
TVL.dtypes

timestamp             int64
tvlUSD                int64
tvlETH              float64
BTC                 float64
ETH                 float64
DAI                 float64
date         datetime64[ns]
dtype: object

### Import the data for Ethereum Prices

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [ ]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv",usecols=(["time","PriceUSD","CapMrktCurUSD","TxTfrValAdjUSD","AdrBalCnt","NVTAdj90","SplyCur","FlowOutExUSD"]))
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time              datetime64[ns]
AdrBalCnt                float64
CapMrktCurUSD            float64
FlowOutExUSD             float64
NVTAdj90                 float64
PriceUSD                 float64
SplyCur                  float64
TxTfrValAdjUSD           float64
dtype: object

### Merge the two Data Frame

In [ ]:
df =df_eth.merge(TVL, left_on='time', right_on='date')
df.head()

,time,AdrBalCnt,CapMrktCurUSD,FlowOutExUSD,NVTAdj90,PriceUSD,SplyCur,TxTfrValAdjUSD,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,date
0,2017-08-02,2563959.0,2.039684e+10,7.162628e+07,20.362229,217.584664,9.374206e+07,6.780871e+08,1501632000,0,0.000000,0.0,0.0,0.0,2017-08-02
1,2017-08-03,2583277.0,2.100563e+10,9.191231e+07,20.979627,224.028898,9.376303e+07,6.062566e+08,1501718400,4,0.018339,0.0,0.0,0.0,2017-08-03
2,2017-08-04,2605667.0,2.077052e+10,7.497462e+07,20.714810,221.471094,9.378433e+07,5.206240e+08,1501804800,4,0.017826,0.0,0.0,0.0,2017-08-04
3,2017-08-05,2624386.0,2.335192e+10,1.051840e+08,23.205816,248.939666,9.380552e+07,6.665590e+08,1501891200,5,0.022665,0.0,0.0,0.0,2017-08-05
4,2017-08-06,2640032.0,2.496085e+10,1.046373e+08,24.667311,266.031146,9.382679e+07,9.229966e+08,1501977600,5,0.019756,0.0,0.0,0.0,2017-08-06


In [ ]:
df=df[["date","tvlUSD","PriceUSD","SplyCur","AdrBalCnt","FlowOutExUSD","TxTfrValAdjUSD"]]

# Test Instrument Validities

Y: "TxTfrValAdjUSD"

X: "FlowOutExUSD"

Z: "SplyCur"

W: "AdrBalCnt"

In [ ]:
df[["FlowOutExUSD","SplyCur"]].corr()

,FlowOutExUSD,SplyCur
FlowOutExUSD,1.000000,0.316229
SplyCur,0.316229,1.000000


# Using the Linear Model Package

https://bashtage.github.io/linearmodels/doc/iv/index.html

In [ ]:
pip install linearmodels

     |████████████████████████████████| 1.5MB 8.2MB/s 
     |████████████████████████████████| 9.5MB 36.1MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
from linearmodels import IV2SLS

### run an OLS

In [ ]:
df['const'] = 1

In [ ]:
ivolsmod = IV2SLS(df.TxTfrValAdjUSD, df[["const","FlowOutExUSD","SplyCur","AdrBalCnt"]], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:         TxTfrValAdjUSD   R-squared:                      0.6481
Estimator:                        OLS   Adj. R-squared:                 0.6473
No. Observations:                1425   F-statistic:                    639.16
Date:                Sun, Jun 27 2021   P-value (F-stat)                0.0000
Time:                        15:49:19   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const         4.405e+10   5.87e+09     7.5032     0.0000   3.254e+10   5.555e+10
FlowOutExUSD     8.5731     0.6533     13.12

### run an IV2SLS

In [ ]:
ivmod = IV2SLS(df.TxTfrValAdjUSD, df[["const","AdrBalCnt"]], df["FlowOutExUSD"], df["SplyCur"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:         TxTfrValAdjUSD   R-squared:                      0.6258
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6253
No. Observations:                1425   F-statistic:                    446.91
Date:                Sun, Jun 27 2021   P-value (F-stat)                0.0000
Time:                        15:49:19   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const        -1.156e+09  1.581e+08    -7.3081     0.0000  -1.466e+09  -8.457e+08
AdrBalCnt        21.195     3.3090     6.405

### comparison of the results

In [ ]:
from collections import OrderedDict
from linearmodels.iv.results import compare

In [ ]:
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
print(compare(res))

                     Model Comparison                     
                                    OLS               2SLS
----------------------------------------------------------
Dep. Variable            TxTfrValAdjUSD     TxTfrValAdjUSD
Estimator                           OLS            IV-2SLS
No. Observations                   1425               1425
Cov. Est.                        robust             robust
R-squared                        0.6481             0.6258
Adj. R-squared                   0.6473             0.6253
F-statistic                      639.16             446.91
P-value (F-stat)                 0.0000             0.0000
==================     ================   ================
const                         4.405e+10         -1.156e+09
                               (7.5032)          (-7.3081)
FlowOutExUSD                     8.5731             11.172
                               (13.123)           (18.721)
SplyCur                         -479.03                 

### Testing endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin’s test is not robust to heteroskedasticity.

In [ ]:
res_2sls.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 30.6350
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f7ef2b5a350

Wooldridge’s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.

In [ ]:
res_2sls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 59.2195
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f7ee981ddd0

### print first stage regression

In [ ]:
print(res_2sls.first_stage)

     First Stage Estimation Results    
                           FlowOutExUSD
---------------------------------------
R-squared                        0.4579
Partial R-squared                0.3467
Shea's R-squared                 0.3467
Partial F-statistic              405.71
P-value (Partial F-stat)         0.0000
Partial F-stat Distn            chi2(1)
==========================  ===========
const                         1.739e+10
                               (20.342)
AdrBalCnt                        81.016
                               (19.687)
SplyCur                         -184.31
                              (-20.142)
---------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


# Using the Statsmodel Package

Not recommended

https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html